<a href="https://colab.research.google.com/github/head1ton/llm_a_z_finetuning/blob/main/notebook/RAG_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

앞으로 뭘하나요?  

1. PDF 파일을 다운로드 합니다.
2. Langchain의 PyPDFLoader로 PDF 파일을 읽어서 페이지 별로 저장합니다.
  - 예를 들어 27페이지의 PDF 파일이라면 27개의 문서를 얻게 됩니다.  
3. 27개의 문서들은 길이가 제각각입니다. 특정 길이(여기서는 500)로 제한하고 더 잘라줍니다.
4. 총 69개의 문서가 나오게 됩니다. 69개의 문서를 전부 임베딩(Embedding)하여 벡터 데이터베이스에 적재합니다.  
5. 벡터 데이터베이스는 Chroma 또는 Faiss를 사용할 것입니다.  
6. 질문을 입력하면 69개의 문서 중 가장 유사도가 높은 문서 4개를 출력합니다.

## 필요한 패키지 다운로드

필요한 패키지를 다운로드 합니다.  

랭체인, 벡터데이터베이스, 임베딩 모델, PDF 파일을 읽는 도구들입니다.

In [1]:
!pip install -qU langchain langchain_openai chromadb transformers sentence-transformers pypdf langchain-community langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 129.4 MB/s et

## 랭체인(Langchain)을 이용한 PDF 로드 및 페이지 별 분할

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings

삼성전자 기업분석 PDF를 다운로드합니다.

In [10]:
!wget https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf

--2025-12-15 09:16:55--  https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf
Resolving wdr.ubion.co.kr (wdr.ubion.co.kr)... 61.100.182.43
Connecting to wdr.ubion.co.kr (wdr.ubion.co.kr)|61.100.182.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253369 (1.2M) [application/pdf]
Saving to: ‘gsat_170823.pdf.1’

gsat_170823.pdf.1   100%[===================>]   1.19M   397KB/s    in 3.1s    

2025-12-15 09:16:59 (397 KB/s) - ‘gsat_170823.pdf.1’ saved [1253369/1253369]



## PDF 페이지 로드

PDF를 Langchain의 PyPDFLoader로 로드하고 분할합니다. 해당 PDF는 참고로 27페이지입니다.  
따라서 문서를 읽고 분할하면 총 27개의 문서가 나옵니다.

In [11]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

27


In [12]:
print(len(pages))

27


첫번째 문서를 출력해봅시다. 랭체인으로 로드한 데이터의 형식은 다음을 따릅니다.  

`Document(metadata={'source': 파일명, 'page': 페이지 번호}, page_content='내용')`

In [13]:
pages[0]

Document(metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등\nIM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품

## 청킹(Chunking): 문서 분할

27개의 문서는 길이가 제각각입니다. 길이 500기준으로 추가적으로 더 쪼개줍니다.  
27개의 문서 중 어떤 문서가 길이가 800이라면 이 문서는 두 개로 다시 잘라서 500이 넘지 않도록 한다는 것입니다.  

추가적으로 문서들을 길이 별로 자르는 것은 `Langchain의 RecursiveCharacterTextSplitter`를 통해서 가능합니다.

이 과정에서 문서는 69개가 됩니다.

In [14]:
## chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splited_docs = text_splitter.split_documents(pages)

print(len(splited_docs))

69


이제 문서는 69개가 되었습니다.

In [15]:
print('69개의 문서 중 첫번째 문서:', splited_docs[0])

69개의 문서 중 첫번째 문서: page_content='2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체
홈페이지 www.samsung.com/sec 구분 전기전자 대기업  
설립일 1961년 07월 01일 대표이사 권오현 
총자산1) 244조 매출액2) 200조
임직원수 95,374명 
∙ 1975년 1월 주식시장 상장
∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 
∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) 
3개의 부문으로 나누어 독립 경영.
부문 제품
CE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}


실제로 길이를 재보면 500이 넘지 않는 것을 확인할 수 있습니다.

In [16]:
print('첫번째 문서의 길이:', len(splited_docs[0].page_content))

첫번째 문서의 길이: 490


69개의 문서를 임베딩해야 합니다. 한글이 가능한 공개되어져 있는 오픈소스 임베딩을 사용해보겠습니다.  

임베딩 모델 위치: https://huggingface.co/BAAI/bge-m3

In [17]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

/tmp/ipython-input-2297800112.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## 크로마(Chroma)

다음은 Langchain에서 Chroma를 이용하여 벡터 DB 사용 시에 가장 간단한 사용법입니다.

```python
# Chroma 를 통해 벡터 저장소 생성. OpenAIEmbeddings()를 사용하거나 HuggingFaceEmbeddings()를 사용하면 된다.
chroma_db = Chroma.from_documents(splited_docs, HuggingFaceEmbeddings())

# 유사도 검색(쿼리)
similar_docs = chroma_db.similarity_search("사용자의 질문")
```

실제로 실행해봅시다.

In [18]:
## Chroma 기반 pdf(docs 벡터화)
db = Chroma.from_documents(splited_docs, model_huggingface)

print('문서의 수:', db._collection.count())

문서의 수: 69


69개의 문서가 위에서 로드한 오픈 소스 임베딩을 바탕으로 임베딩되었다고 보면 됩니다.  

임의로 '삼성전자의 주요 사업영역은?' 이라는 질문을 던지고 임베딩 벡터의 유사도 기준으로 상위 4개의 문서를 출력해봅시다.

In [19]:
## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db.similarity_search(question, k=4)

상위 4개의 문서를 출력해보겠습니다.

In [20]:
print('검색된 문서의 수:', len(docs))

검색된 문서의 수: 4


In [21]:
for doc in docs:
  print(doc)
  print('--' * 100)

page_content='11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공' metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'page': 9, 'moddate': '2017-08-16T00:21:02-08:00', 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'total_pages': 27}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 벡터 데이터베이스 로컬에 파일로 저장

vectorize된 텍스트를 Chroma db 파일로 저장하기

In [22]:
db_toFiles = Chroma.from_documents(splited_docs, model_huggingface, persist_directory = './samsumg.db')
print('문서의 수:', db_toFiles._collection.count())

문서의 수: 138


이 시점에 현재 경로에 samsung.db라는 파일이 생깁니다.

In [23]:
# 현재 경로에 있는 파일들 확인
%ls -al

total 2468
drwxr-xr-x 1 root root    4096 Dec 15 09:16 ./
drwxr-xr-x 1 root root    4096 Dec 15 09:02 ../
drwxr-xr-x 4 root root    4096 Dec  9 14:41 .config/
-rw-r--r-- 1 root root 1253369 Aug 23  2017 gsat_170823.pdf
-rw-r--r-- 1 root root 1253369 Aug 23  2017 gsat_170823.pdf.1
drwxr-xr-x 1 root root    4096 Dec  9 14:42 sample_data/
drwxr-xr-x 3 root root    4096 Dec 15 09:17 samsumg.db/


저장했던 db 파일을 읽어와서 동일한 실습을 진행하는 것도 가능합니다.

In [24]:
db_fromfile = Chroma(persist_directory = './samsumg.db',embedding_function=model_huggingface)
print('문서의 수:', db_fromfile._collection.count())

## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db_fromfile.similarity_search(question, k=4)

print('검색된 문서의 수:', len(docs))

문서의 수: 138
검색된 문서의 수: 4


유사도 점수를 함께 출력하거나 상위 문서의 개수를 조절하는 것도 가능합니다.

In [25]:
question = '삼성전자의 주요 사업영역은?'

# TOP 3개만 추출
docs3 = db_fromfile.similarity_search_with_relevance_scores(question, k=3)

In [26]:
for doc in docs3:
  print(doc)
  print('--' * 100)

(Document(id='61e71d3d-cf13-4f7c-a4f0-48f75d46ff4e', metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'creator': 'nPDF (pdftk 1.41)', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'page_label': '10', 'page': 9, 'total_pages': 27, 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00'}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문\n① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공'), 0.6830467219193441)
---------------------------------------------------------------------------

위의 검색 결과의 맨 오른쪽을 보면 유사도 점수가 측정되어 출력됩니다.  


**주의사항**  
이 유사도 점수를 일종의 임계값으로 사용하는 분들도 있는데 사실 그다지 정확도가 보장되는 방법은 아닙니다.  

이 임베딩 모델은 여러분들이 실습하고 있는 도메인 데이터에 특화된 임베딩 모델이 아니기 때문입니다.  
임베딩 모델을 해당 도메인으로 파인 튜닝 했을 때 의미가 있습니다.

유사도가 가장 높은 문서와 유사도 점수를 출력해보겠습니다.

In [27]:
print(f"가장 유사한 문서 : {docs3[0][0].page_content}")
print('*'*20)
print(f"가장 유사한 문서의 문서의 유사도 : {docs3[0][1]}")

가장 유사한 문서 : 11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공
********************
가장 유사한 문서의 문서의 유사도 : 0.6830467219193441


## GPT-4 호출하기

이제 앞서 발급받았던 OpenAI key 값을 입력하여 GPT-4를 사용해봅시다.

In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
# from langchain.chains import RetrievalQA
import os

In [29]:
# os.environ['OPENAI_API_KEY'] =  "여러분들의 Key 값"

이제 GPT-4에 전달할 프롬프트를 작성합니다.

In [30]:
import os
from typing import Literal

# from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# load_dotenv()

from google.colab import userdata


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    # api_key = os.getenv("OPENROUTER_API_KEY")
    api_key = userdata.get('OPENROUTER_API_KEY')

    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    openai_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
        **kwargs,
    }
    if max_tokens is not None:
        openai_kwargs["max_tokens"] = max_tokens
    if base_url:
        openai_kwargs["base_url"] = base_url
    return ChatOpenAI(**openai_kwargs)

def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict = {}
    embedding_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: Literal["float", "base64"] = "float",
    input_text: str | list[str] = "",
    **kwargs,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

In [31]:
from langchain_core.messages import SystemMessage, HumanMessage
# from src.llm.client import create_openrouter_llm

# LLM은 한 번만 생성해서 재사용 (중요)
llm = create_openrouter_llm(
    model="openai/gpt-oss-20b:free",  # 또는 openai/gpt-4o, gpt-oss-20b 등
    temperature=0
)

In [47]:
# GPT-4에 전달할 프롬프트
template = """당신은 삼성전자 기업 보고서를 설명해주는 챗봇 '삼성맨'입니다.
안상준 개발자가 만들었습니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 이모지를 사용하며 친근하게 답변하세요.
{context}

Question: {input}
Answer:
"""

# 프롬프트 템플릿은 {question}에는 사용자의 질문이 들어간다. {context}에는 사용자의 질문에 따른 검색 결과가 들어간다.
prompt = PromptTemplate.from_template(template)

In [48]:
# # GPT-4 선언. 단, API Key 값이 셋팅되어져 있어야 실행 가능합니다.
# llm = ChatOpenAI(model_name="gpt-4o")

ChatOpenAI로부터 llm 객체를 선언하여 GPT-4 호출이 가능합니다.

In [49]:
result = llm.invoke('안녕 반가워 GPT-4야 나는 안상준이라고해')
print(result.content)

안녕하세요, 안상준님! 반갑습니다. 😊  
무엇을 도와드릴까요? 궁금한 점이나 필요한 정보가 있으면 언제든 말씀해 주세요!


## 벡터 데이터베이스와 LLM의 연결

`RetrievalQA.from_chain_type()`

- llm: 앞서 선언한 llm 객체를 넣어주세요.
- retriever: 앞서 선언한 retriever 객체를 넣어주세요.
- chain_type_kwargs={"prompt": ?}: 앞서 선언한 프롬프트 템플릿을 넣어주세요.

In [50]:
# 벡터 데이터베이스를 LLM과 연결하기 위해서 retriever 객체로 선언
retriever = db.as_retriever(search_kwargs={"k": 3})

In [51]:
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm, # LLM 연결
#     retriever=retriever, # 리트리버(벡터 데이터베이스) 연결
#     chain_type_kwargs={"prompt": prompt}, # 프롬프트 템플릿 연결
#     return_source_documents=True # 실제 검색된 문서도 확인하려면 True
#     )

In [52]:
!pip install langchain==1.0.5

In [53]:
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

llm_prompt_chain = create_stuff_documents_chain(
    llm,
    prompt
)

qa_chain = create_retrieval_chain(retriever, llm_prompt_chain)

In [54]:
input_text = "삼성전자의 주요 사업영역은?"
chatbot_response = qa_chain.invoke({"input": input_text})

In [55]:
chatbot_response

{'input': '삼성전자의 주요 사업영역은?',
 'context': [Document(id='2cfbb349-3d64-4688-bb72-181287711fe6', metadata={'total_pages': 27, 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'page_label': '10', 'page': 9, 'creationdate': '2017-08-16T00:21:02-08:00'}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문\n① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공'),
  Document(id='0aa8a2a9-1b82-42a6-9811-c168a0dd31e6', 

In [56]:
print(chatbot_response["answer"])

삼성전자의 주요 사업 영역은 **3개 부문**으로 나뉘어 있어요! 🚀

| 부문 | 주요 제품·서비스 | 핵심 포인트 |
|------|------------------|-------------|
| **CE (Consumer Electronics)** | TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등 | 생활가전·디스플레이 혁신으로 세계 1위 도전 |
| **IM (Information Technology & Mobile Communications)** | 무선 통신, 스마트폰, 모바일 기기 | 인간 중심 혁신으로 소비자 편의 제공 |
| **DS (Device Solutions)** | 반도체, 메모리, 시스템 반도체 등 | 첨단 디바이스 솔루션으로 글로벌 시장 선도 |

이렇게 세 부문이 독립적으로 운영되면서 삼성전자는 휴대폰, IT 기기, 반도체, TV 등 다양한 분야에서 세계적인 제조업체로 자리매김하고 있어요. 🌍✨


In [59]:
print('내가 넣었던 질문:', chatbot_response['input'])

내가 넣었던 질문: 삼성전자의 주요 사업영역은?


In [61]:
print('검색 결과를 바탕으로 GPT-4가 작성한 답변:', chatbot_response['answer'])

검색 결과를 바탕으로 GPT-4가 작성한 답변: 삼성전자의 주요 사업 영역은 **3개 부문**으로 나뉘어 있어요! 🚀

| 부문 | 주요 제품·서비스 | 핵심 포인트 |
|------|------------------|-------------|
| **CE (Consumer Electronics)** | TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등 | 생활가전·디스플레이 혁신으로 세계 1위 도전 |
| **IM (Information Technology & Mobile Communications)** | 무선 통신, 스마트폰, 모바일 기기 | 인간 중심 혁신으로 소비자 편의 제공 |
| **DS (Device Solutions)** | 반도체, 메모리, 시스템 반도체 등 | 첨단 디바이스 솔루션으로 글로벌 시장 선도 |

이렇게 세 부문이 독립적으로 운영되면서 삼성전자는 휴대폰, IT 기기, 반도체, TV 등 다양한 분야에서 세계적인 제조업체로 자리매김하고 있어요. 🌍✨


In [62]:
print('챗봇이 참고한 실제 검색 결과에 해당하는 유사도 상위 3개 문서:')

for doc in chatbot_response['context']:
  print(doc)
  print('--' * 100)

챗봇이 참고한 실제 검색 결과에 해당하는 유사도 상위 3개 문서:
page_content='11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공' metadata={'total_pages': 27, 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'page_label': '10', 'page': 9, 'creationdate': '2017-08-16T00:21:02-08:00'}
-------------------------------------------------------------------------------------------------------------------------------

In [63]:
input_text = "25년 2월의 서울 아파트 집값은?"
chatbot_response = qa_chain.invoke({"input": input_text})

In [64]:
chatbot_response['answer']

'죄송해요! 😔 제공해 주신 검색 결과에는 2025년 2월 서울 아파트 가격에 대한 정보가 없어요. 그래서 답변을 드릴 수 없답니다. 다른 궁금한 점이 있으면 언제든 물어봐 주세요! 🚀'

In [43]:
# input_text = "삼성전자의 주요 사업영역은?"
# chatbot_response = qa_chain.invoke(input_text)

In [ ]:
# chatbot_response

{'query': '삼성전자의 주요 사업영역은?',
 'result': "안녕하세요! 😊 '삼성맨'입니다. 삼성전자의 주요 사업영역에 대해 설명해드릴게요. 삼성전자는 세 가지 주요 사업부문으로 나뉘어 독립 경영을 하고 있습니다:\n\n1. **Consumer Electronics (CE) 부문**: 이 부문에서는 TV, 모니터, 냉장고, 세탁기, 에어컨, 의료기기 등 다양한 전자제품을 생산합니다. 특히 스마트TV와 초대형 프리미엄 TV 전략을 통해 세계 시장을 선도하고 있죠.\n\n2. **Information technology & Mobile communication (IM) 부문**: 이 부문은 주로 휴대폰과 관련된 정보기술 및 모바일 통신 제품을 다루며, 혁신적인 기술을 통해 사용자에게 새로운 가치를 제공하고 있습니다.\n\n3. **Device Solutions (DS) 부문**: 이 부문은 반도체와 같은 장치 솔루션을 포함하며, 첨단 기술을 기반으로 한 제품들을 생산하고 있습니다.\n\n삼성전자는 이렇게 다양한 사업부문을 통해 글로벌 시장에서 활발히 활동하고 있습니다! 📱🏠💻",
 'source_documents': [Document(metadata={'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성

In [58]:
# print('내가 넣었던 질문:', chatbot_response['query'])

In [ ]:
# print('검색 결과를 바탕으로 GPT-4가 작성한 답변:', chatbot_response['result'])

검색 결과를 바탕으로 GPT-4가 작성한 답변: 안녕하세요! 😊 삼성맨입니다.

삼성전자의 주요 사업 영역은 크게 세 가지로 나누어집니다:

1. **Consumer Electronics (CE) 부문**:
   - **영상디스플레이**: 스마트TV와 초대형 프리미엄 TV에서 세계 1위를 목표로 하고 있어요.
   - **생활가전**: 혁신적인 기술과 가치를 통해 생활과 문화를 변화시키고 있습니다.
   - **의료기기**: 정확하고 빠른 진단을 돕는 혁신적인 의료기기를 개발 중이에요.

2. **Information Technology & Mobile Communication (IM) 부문**:
   - **무선**: 인간 중심의 혁신을 통해 소비자에게 새로운 가치와 편의를 제공하기 위해 노력하고 있습니다.

3. **Device Solutions (DS) 부문**:
   - 이 부문은 주로 반도체 및 디지털 솔루션과 관련된 산업을 포함하고 있습니다.

이 외에 다른 구체적인 정보가 필요하시면 알려주세요! 🤗


In [ ]:
# print('챗봇이 참고한 실제 검색 결과에 해당하는 유사도 상위 3개 문서:')

# for doc in chatbot_response['source_documents']:
#   print(doc)
#   print('--' * 100)

챗봇이 참고한 실제 검색 결과에 해당하는 유사도 상위 3개 문서:
page_content='11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공' metadata={'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27}
-------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# input_text = "25년 2월의 서울 아파트 집값은?"
# chatbot_response = qa_chain.invoke(input_text)

In [ ]:
# chatbot_response['result']

'죄송하지만, 검색 결과에 2025년 2월의 서울 아파트 집값에 대한 정보가 없습니다. 😅 다른 질문이 있으시면 말씀해 주세요!'

## 성능 향상을 위한 고민 포인트

시간의 한계로 인해 여기서 자세히 다루지는 못하지만 여러분들이 앞으로 공부하는 방향에 있어서 키워드들을 제시하고자 합니다.

**Q. PDF를 페이지 별로 로드하고 또 그 페이지들을 적당한 길이로 자르면 각각의 문서들은 내용이 앞이나 뒤에 문맥이 없이 중간에 끊길텐데 이대로 검색 문서에 적재되어도 괜찮을까요?**

A) 당연히 성능에 영향을 줍니다. Chunking에 대한 고민을 많이 해야 하고 이를 위한 다양한 검색 알고리즘과 청킹 알고리즘들이 존재하여 이에 대해서 공부하고 적용해야 합니다.  

아래 질문과 연계된 질문이기도 하므로 아래 질문과 맞추어서 보충 답변을 해보겠습니다.

**Q. 사용자의 질문에 대한 답이 검색된 문서에 없을 경우에는 어떻게 해야할까요?**

A) RAG에서는 기본적으로 검색된 문서에 대한 답이 없을 때에 대한 시나리오를 따로 준비해야 합니다. 여기서는 검색 결과가 없다라고 단순하게 답변하도록 유도하였습니다.  

실제로 질문이 갖고 있는 문서로 답변할 수 없는 경우인지 원래 검색이 되어야 하는데 검색 성능이 안 좋아서 검색이 안 된 건지를 알아야 합니다. 만약 후자의 경우라면 검색 성능을 높이기 위해서 다음과 같은 고민들을 해야 합니다.  

- 문서 Chunking을 잘못해서 임베딩 성능이 좋지 않지는 않은가?
  - semantic chunking, 합성 데이터 방식(요약 등), 앤트로픽의 Contextual Retrieval을 고민해볼까?
- 임베딩 모델의 성능이 좋지 않아서 파인 튜닝을 해볼까?  
- reranking이라고 하는 2단계 검색 과정을 넣어서 검색 성능을 높여야 할까?  
- BM25와 같은 전통적인 키워드 기반의 검색 알고리즘을 혼합해볼까?  
- HyDE와 같은 검색 성능을 높이는 방법을 적용해볼까?  



**Q. PDF 파일에 이미지나 테이블이 있었을텐데 이런 건 제대로 검색이 되고 또 답변할 수 있을까요?**

A) 멀티모달 RAG의 영역입니다. VLM과 같은 멀티모달 LLM, 멀티모달 임베딩, 또는 OCR이 가능한 패키지인 unstructured.io 등을 사용하여 다양한 방법으로 이를 처리할 수 있습니다.

**Q. 멀티턴, 멀티쿼리 등과 같은 복잡한 쿼리에 대해서도 대응 가능할까?**

A) 멀티턴은 현 구조에서도 조금의 수정을 통해 가능하지만, 멀티 쿼리(A and B)는 현 구조에서는 어렵습니다. 이 경우 에이전트 RAG를 수행하거나 앞단에 별도의 모듈을 두어서 처리해야 합니다.

# Task
Re-run the code in cell `ThRUzh_DN85W` to import LangChain chain functionalities and construct the RAG chain.

## Execute cell ThRUzh_DN85W

### Subtask:
Re-run the cell that was previously causing a ModuleNotFoundError to confirm if the reinstallation of LangChain has resolved the issue. This cell imports necessary chain functionalities and constructs the RAG chain.


## Summary:

### Data Analysis Key Findings
*   The code in cell `ThRUzh_DN85W` was re-executed to confirm if a `ModuleNotFoundError` related to LangChain imports had been resolved following a reinstallation of the library. This cell is critical for importing chain functionalities and constructing the RAG chain.

### Insights or Next Steps
*   Verify the successful execution of cell `ThRUzh_DN85W` to ensure that LangChain components are now correctly imported and the RAG chain can be constructed.
*   If the `ModuleNotFoundError` recurs, a deeper investigation into the Python environment, package dependencies, or the LangChain installation process will be necessary.
